In [199]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image 

# 1. Read in the training/testing data, transform them into pixelation vectors

In [200]:
#Read the list for training/testing data points: 
f_train = open('downgesture_train.list','r')
f_test = open('downgesture_test.list','r')
line_train = f_train.readlines()
line_test  = f_test.readlines()

In [320]:
#Lists for storing training data/testing data:
training = []
testing = []
#Lists for storing labels of training/testing data
training_label = []
testing_label = []

#Store the training data to list 'training' with pixelation
for i in line_train:
    im_i = np.asarray(Image.open('C:/Users/User/Desktop/Machine Learning HW 5/'+(i.rstrip('\n'))))
    training.append(im_i)
    
#Storing the labels of respective data poins in the training set     
    if 'down' in i:
        training_label.append([1])
    else:
        training_label.append([0])

#Store the testing data to list 'testing' with pixelation
for j in line_test:
    im_j = np.asarray(Image.open('C:/Users/User/Desktop/Machine Learning HW 5/'+(j.rstrip('\n'))))
    testing.append(im_j)
    
#Storing the labels of respective data poins in the testing set       
    if 'down' in j:
        testing_label.append([1])
    else:
        testing_label.append([0])
    
f_train.close()
f_test.close()

In [240]:
#Reshape each image's pixel data to a 960 x 1 vector
for i in range(len(testing)):
    testing[i]=testing[i].reshape(960,1)
    
for j in range(len(training)):
    training[j]=training[j].reshape(960,1)

In [241]:
#Check the data:
#len(training)
#len(testing)
#training[5]
#training[5].size
#testing[10]
#testing[10].size

# Define the neural networks model

In [242]:
class neural_networks(object):
    
#Randomize the initial weights and bias:     
    def __init__(self,structure): #structure = a list containing the number of neurons in each layer 
        self.structure = structure
        self.weight=[]
        self.bias=[]
        self.delta_weight = []
        self.delta_bias = []
        self.length = len(structure)
                
        for i,j in zip(self.structure[:-1],self.structure[1:]):
            (self.weight).append(np.random.uniform(-0.01,0.01,size=(j,i)))   
            #Here, self.weights[i] stores the weights connecting the i+1 and i+2 layer of networks 

            
        for i in structure[1:]:
            (self.bias).append(np.random.randn(i,1))
            #Here, self.bias[i] stores the biases in the i+1 layer of networks (exclude the input layer)
            
            
        for i,j in zip(self.structure[:-1],self.structure[1:]):
            (self.delta_weight).append(np.zeros((j,i)))   
          
            
        for i in structure[1:]:
            (self.delta_bias).append(np.zeros((i,1)))

#Define the sigmoid function:
    def sigmoid(self,s): 
        sig = 1/(1 + np.exp(-s))
        return sig 

#Define the function for sigmoid function's derivative:    
    def del_sigmoid(self,s):
        del_sig = self.sigmoid(s)*(1-self.sigmoid(s))
        return del_sig
    
#Forward process:
    def feedforward (self,x):
        
        self.zs=[]   
        self.activations=[]
        for i in range(self.length-1):
            self.zs.append([])           #A list for containing w*x + b among different layers 
            self.activations.append([])  #A list for containing  all activation functions among different layers
            
        self.activations.insert(0,x)     #Add the input variable to the first column of activations list, for later computation usage
       
    
       #Calculate the activation functions among respective layers: 
        n=0    
        while n < (self.length-1):
        
            for w,b in zip(self.weight[n],self.bias[n]):
                z = np.dot(w,x)+b 
                self.zs[n].append(z)
                self.activations[n+1] = list(map(self.sigmoid,self.zs[n]))
            x = self.zs[n] 
            n=n+1     

            
#Backward process:
    def backpropagation(self,desired_output):
        
        #del c/del a :
        del_cost = [[2*(desired_output[i]-self.activations[-1][i]) for i in range(self.structure[-1])]] 
        #(del c/del a)       
        #(del a/del z):
        del_cost_sigmoid = [self.del_sigmoid(self.zs[-1][i]) for i in range(len(self.zs[-1]))]
        
        delta = [(del_cost[i] * del_cost_sigmoid[i]) for i in range(len(del_cost))]  
        self.delta_bias[-1] = delta
        self.delta_weight[-1] = np.dot(delta,np.array(self.activations[-2]).T)
        
        
        #Aftering computing the gradients of hidden layers-output layers, next, calculate the gradients in the remaining layers:
        for layer in range(2,self.length):
            
            self.da_dz = list(map(self.del_sigmoid,self.zs[-layer]))
            delta = [(np.dot(np.array(self.weight[-layer+1]).T,delta))[i]*self.da_dz[i] for i in range(len(self.da_dz))]
            self.delta_bias[-layer] = delta
            self.delta_weight[-layer] = np.dot(delta,np.array(self.activations[-layer-1]).T)
        
        return self.delta_bias,self.delta_weight

# Train the model 

In [315]:
model = neural_networks([960,100,1])

In [312]:
b_sum = [[0]*100,[0]*1]         #A list for storing the sum of bias gradients after training through each data point 
w_sum = [[[0]*960]*100,[[0]*100]*1]  #A list for storing the sum of weight gradients training through each data point

for _ in range(1000):           #Updating the gradients with 1000 iterations 
    for x,y in zip(training,training_label):
        model.feedforward(x)
        bn,wn = model.backpropagation(y)
    
    #The following four for loops calculate the sum of gradients among 184 training data points
        for bias_layer1 in range(100):
            b_sum[0][bias_layer1] = b_sum[0][bias_layer1]+bn[0][bias_layer1]
    
        for bias_layer2 in range(1):
            b_sum[1][bias_layer2] = b_sum[1][bias_layer2]+bn[1][0][bias_layer2]
    
        for neural_1 in range(100):
            for w_layer1 in range(960):
                w_sum[0][neural_1][w_layer1] = w_sum[0][neural_1][w_layer1]+wn[0][neural_1][0][w_layer1]
    
        for neural_2 in range(1):
            for w_layer2 in range(100):
                w_sum[1][neural_2][w_layer2] = w_sum[1][neural_2][w_layer2]+wn[1][0][neural_2][w_layer2]
   
   #Next, implement the gradient descend,updating the weights/biases with learning rate 0.1 
    for i in range(100):
        model.bias[0][i] = model.bias[0][i] - (0.1*((b_sum[0][i])/184))

    for j in range(1):
        model.bias[1][j] = model.bias[1][j] - (0.1*((b_sum[1][j])/184))

    for k in range(100):
        for l in range(960):
            model.weight[0][k][l] = model.weight[0][k][l] - (0.1*((wn[0][k][0][l])/184))
        
    for m in range(1):
        for n in range(100):
            model.weight[1][m][n] = model.weight[1][m][n] - (0.1*((wn[1][0][m][n])/184))

# Testing model

In [316]:
#Testing 

prediction = []
accurate_count = 0 

for x,y in zip(testing,testing_label):
    model.feedforward (x)
    if model.activations[-1][0][0] >= 0.5:
        prediction.append(1)
        if y[0] == 1:
            accurate_count += 1
        else:
            pass
        
    elif model.activations[-1][0][0] < 0.5:
        prediction.append(0)
        if y[0] == 0:
            accurate_count +=1
        else:
            pass

In [317]:
prediction.reshape  #Print the prediction result

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [318]:
accuracy = accurate_count/len(testing)  #accuracy rate 
print(accuracy)

0.7831325301204819
